# Create some test data
### Join GTFS stop points to stop times

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString
from scipy.spatial import cKDTree 
import os

import importlib
# local modules
import parse_functions
importlib.reload(parse_functions)

%matplotlib inline

In [2]:
gtfs_dir = '../utah-transit-authority_20160331_1817'
routes = pd.read_csv(os.path.join(gtfs_dir, 'routes.txt'))
shapes = pd.read_csv(os.path.join(gtfs_dir, 'shapes.txt'))
trips = pd.read_csv(os.path.join(gtfs_dir, 'trips.txt'))
stops = pd.read_csv(os.path.join(gtfs_dir, 'stops.txt'))
stop_times = pd.read_csv(os.path.join(gtfs_dir, 'stop_times.txt'), dtype={'stop_id': object})
calendar = pd.read_csv(os.path.join(gtfs_dir, 'calendar.txt'))

In [3]:
# Speedy Spatial

def ckdnearest(gdA, gdB, bcol):
    """
    Get bcol for from every nearest point in gdA.
    See: https://gis.stackexchange.com/questions/222315/geopandas-find-nearest-point-in-other-dataframe
    """
    import time
    t = time.time()
    nA = np.array(list(zip(gdA.geometry.x, gdA.geometry.y)) )
    nB = np.array(list(zip(gdB.geometry.x, gdB.geometry.y)) )
    print('arrays', time.time() - t)

    t = time.time()
    btree = cKDTree(nB)
    print('tree', time.time() - t)

    t = time.time()
    dist, idx = btree.query(nA,k=1)
    print('query', time.time() - t)

    t = time.time()
    df = pd.DataFrame.from_dict({'distance': dist.astype(int),
                                 'aindex': gdA.index,
                                 'bcol_' + bcol : gdB.iloc[idx, gdB.columns.get_loc(bcol)].values })
    df.set_index('aindex', inplace=True)
    print('dataframe', time.time() - t)
    return df

#nearest_df = ckdnearest(subset_stops, subset_stops, 'stop_name')

In [4]:
stop_points = parse_functions.transform_xy_to_points(stops, 'stop_lon', 'stop_lat', 'stop_id')
stop_points.crs = {'init' :'epsg:4326'}
stop_points['geometry'] = stop_points['geometry'].to_crs(epsg=3857)
stop_points.head(3)

,stop_lat,stop_code,stop_lon,stop_url,parent_station,stop_desc,stop_name,location_type,zone_id,geometry,stop_id
stop_id,,,,,,,,,,,
11541,41.166905,629050,-111.962747,NaN,NaN,E 5300 S,5300 S @ 711 E,0,NaN,POINT (-12463635.98385612 5036991.371158207),11541
11546,41.206463,623264,-111.970663,NaN,NaN,S WASHINGTON BLVD,WASHINGTON BLVD @ 3180 S,0,NaN,POINT (-12464517.18894524 5042842.771688678),11546
11544,41.199655,623261,-111.970843,NaN,NaN,S WASHINGTON BLVD,WASHINGTON BLVD @ 3500 S,0,NaN,POINT (-12464537.22645358 5041835.483675041),11544


In [5]:
shape_points = parse_functions.transform_xy_to_points(shapes, 'shape_pt_lon', 'shape_pt_lat', 'shape_id')
shape_points.crs = {'init' :'epsg:4326'}
shape_points['geometry'] = shape_points['geometry'].to_crs(epsg=3857)
shape_points.head(3)


,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled,geometry,shape_id
shape_id,,,,,,
133799,40.390900,-111.577479,1,0.0000,POINT (-12420748.14627717 4922910.465132287),133799
133799,40.390810,-111.577240,2,0.0226,POINT (-12420721.54091888 4922897.310976486),133799
133799,40.390729,-111.577080,3,0.0388,POINT (-12420703.72980035 4922885.472251293),133799


In [6]:
nearest_df = ckdnearest(stop_points, shape_points, 'geometry')

arrays 8.315207481384277
tree 0.2724027633666992
query 0.02760004997253418
dataframe 0.0040514469146728516


In [7]:
nearest_df.head()

,bcol_geometry,distance
aindex,,
11541,POINT (-12463665.2608822 5037015.770540411),38
11546,POINT (-12464532.43971548 5042783.142291525),61
11544,POINT (-12464546.91124928 5041849.538823912),17
11548,POINT (-12464515.74179186 5043703.517820037),52
11549,POINT (-12464505.72303769 5044322.083014605),53


In [104]:
stop_points.size

68838

In [106]:
shape_points.size

2152512

In [76]:
nearest_df.sort_values(by=['distance'], ascending=False).head()

AttributeError: 'numpy.ndarray' object has no attribute 'sort_values'

In [35]:
nearest_df['distance'].sum()

0

In [36]:
nearest_df.groupby('b_indx')['distance'].sum().sort_values(ascending=False)

b_indx
358743    0
131454    0
131507    0
131622    0
131769    0
131850    0
131852    0
131919    0
132020    0
132021    0
132513    0
132911    0
132938    0
133039    0
133264    0
133312    0
133555    0
131489    0
131427    0
133572    0
131389    0
130564    0
130686    0
130700    0
130738    0
130780    0
130783    0
130842    0
130890    0
130924    0
         ..
248199    0
248205    0
248210    0
248246    0
248255    0
248256    0
248260    0
248303    0
248345    0
248364    0
248393    0
247634    0
247617    0
247591    0
247349    0
246465    0
246468    0
246489    0
246521    0
246636    0
247299    0
247381    0
247589    0
247398    0
247453    0
247463    0
247469    0
247536    0
247544    0
428       0
Name: distance, Length: 4860, dtype: int32

# Make stop time points

In [37]:
stop_time_points = stop_times.join(stop_points, on='stop_id', lsuffix='_time', rsuffix='_stops')
len(stop_time_points.loc[stop_time_points['stop_id_stops'].isnull()])

0

In [38]:
stop_time_points.head()

,trip_id,arrival_time,departure_time,stop_id_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,...,stop_code,stop_lon,stop_url,parent_station,stop_desc,stop_name,location_type,zone_id,geometry,stop_id_stops
0,2729438,10:13:00,10:13:00,23259,1,NaN,NaN,NaN,NaN,NaN,...,198079,-112.004673,NaN,NaN,W OLD BINGHAM HWY,OLD BINGHAM HWY @ 4786 W,0,NaN,POINT (-112.004673 40.581717),23259
1,2729438,10:13:30,10:13:30,22847,2,NaN,NaN,NaN,NaN,NaN,...,101869,-112.005077,NaN,NaN,S 4800 W,4800 W @ 9253 S,0,NaN,POINT (-112.005077 40.583308),22847
2,2729438,10:14:16,10:14:16,22831,3,NaN,NaN,NaN,NaN,NaN,...,101856,-112.005087,NaN,NaN,S 4800 W,4800 W @ 9049 S,0,NaN,POINT (-112.005087 40.586969),22831
3,2729438,10:14:33,10:14:33,23257,4,NaN,NaN,NaN,NaN,NaN,...,198077,-112.005766,NaN,NaN,W 9000 S,9000 S @ 4820 W,0,NaN,POINT (-112.005766 40.588034),23257
4,2729438,10:15:50,10:15:50,23255,5,NaN,NaN,NaN,NaN,NaN,...,198075,-112.013903,NaN,NaN,W 9000 S,9000 S @ 5156 W,0,NaN,POINT (-112.013903 40.588043),23255


In [39]:
stop_time_points = gpd.GeoDataFrame(stop_time_points, geometry='geometry')
stop_time_points.crs = {'init' :'epsg:4326'}

In [40]:
len(stop_time_points)

1515685

In [41]:
# stop_time_points.to_file('data/stop_time_points.shp')

In [42]:
subset_size = 1000
subset_stops = stop_time_points.head(subset_size).copy()
# big_geo = subset_stops.geometry.unary_union

# Make stop to shape connections

In [43]:
len(stop_points)

6258

In [44]:
len(shapes.groupby('shape_id')) * len(stop_points)

C:\Users\kwalker\AppData\Local\ESRI\conda\envs\geofun\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 'shape_id' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.


5438202

In [45]:
trip_stop_grp = stop_times.groupby(['stop_id', 'trip_id'])

In [46]:
len(trip_stop_grp)

1514191

In [47]:
len(stop_times)

1515685

In [48]:
stop_times.set_index('trip_id', inplace=True)

In [52]:
stop_times.index

Int64Index([2729438, 2729438, 2729438, 2729438, 2729438, 2729438, 2729438,
            2729438, 2729438, 2729438,
            ...
            2729431, 2729431, 2729431, 2729431, 2729431, 2729431, 2729431,
            2729431, 2729431, 2729431],
           dtype='int64', name='trip_id', length=1515685)

In [56]:
trips.set_index('trip_id', inplace=True)

In [57]:
#trips.set_index('trip_id', inplace=True)
stop_shapes = stop_times.join(trips, lsuffix='_times', rsuffix='_trips')
stop_shapes.head()

,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,block_id,route_id,direction_id,trip_headsign,shape_id,service_id
trip_id,,,,,,,,,,,,,,,
2507949,05:55:00,05:55:00,24384,1,NaN,NaN,NaN,NaN,NaN,a_863938,60966,0,University Medical Center,132426,4_merged_2787244
2507949,05:59:16,05:59:16,5479,2,NaN,NaN,NaN,NaN,NaN,a_863938,60966,0,University Medical Center,132426,4_merged_2787244
2507949,05:59:49,05:59:49,24221,3,NaN,NaN,NaN,NaN,NaN,a_863938,60966,0,University Medical Center,132426,4_merged_2787244
2507949,06:00:36,06:00:36,5481,4,NaN,NaN,NaN,NaN,NaN,a_863938,60966,0,University Medical Center,132426,4_merged_2787244
2507949,06:01:35,06:01:35,13105,5,NaN,NaN,NaN,NaN,NaN,a_863938,60966,0,University Medical Center,132426,4_merged_2787244


In [58]:
len(stop_shapes.groupby('shape_id'))

736

In [59]:
stop_shapes.groupby(['stop_id', 'shape_id']).size()

stop_id  shape_id
1022     132426       38
         132427       36
         132430      134
         132431      132
         132444       74
         132445       74
         132469       24
         132476        6
         132477        6
         132487        6
         132488        6
         132561       14
         132562        2
         132563       14
         132588       11
         132589       82
         132590       11
         132591       82
         139173       19
         139174       18
         139177      115
         139178      114
         139191       81
         139192       81
         139216       12
         139223        3
         139224        3
         139234        3
         139235        3
         139306        7
                    ... 
9612     132471       42
         132476        6
         132514        8
         132517       12
         132518       22
         139207       23
         139217       16
         139218       67
       

# Speedy Spatial

In [26]:
 def ckdnearest(gdA, gdB, bcol):
    """
    Get bcol for from every nearest point in gdA.
    See: https://gis.stackexchange.com/questions/222315/geopandas-find-nearest-point-in-other-dataframe
    """
    import time
    t = time.time()
    nA = np.array(list(zip(gdA.geometry.x, gdA.geometry.y)) )
    nB = np.array(list(zip(gdB.geometry.x, gdB.geometry.y)) )
    print('arrays', time.time() - t)
    
    t = time.time()
    btree = cKDTree(nB)
    print('tree', time.time() - t)
    
    t = time.time()
    dist, idx = btree.query(nA,k=2)
    print(dist)
#     print('query', time.time() - t)
    
#     t = time.time()
#     df = pd.DataFrame.from_dict({'distance': dist.astype(int),
#                              'bcol' : gdB.loc[idx, bcol].values })
#     print('dataframe', time.time() - t)
#     return df

nearest_df = ckdnearest(subset_stops, subset_stops, 'stop_name')
# nearest_df.head()

arrays 0.08613824844360352
tree 0.0010581016540527344
[[0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 ...
 [0.         0.00278604]
 [0.         0.00278604]
 [0.         0.00238059]]


In [27]:
subset_stops.index

RangeIndex(start=0, stop=1000, step=1)

In [16]:
stop_time_points.head(100)

,trip_id,arrival_time,departure_time,stop_id_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,...,stop_code,stop_lon,stop_url,parent_station,stop_desc,stop_name,location_type,zone_id,geometry,stop_id_stops
0,2729438,10:13:00,10:13:00,23259,1,NaN,NaN,NaN,NaN,NaN,...,198079,-112.004673,NaN,NaN,W OLD BINGHAM HWY,OLD BINGHAM HWY @ 4786 W,0,NaN,POINT (-112.004673 40.581717),23259
1,2729438,10:13:30,10:13:30,22847,2,NaN,NaN,NaN,NaN,NaN,...,101869,-112.005077,NaN,NaN,S 4800 W,4800 W @ 9253 S,0,NaN,POINT (-112.005077 40.583308),22847
2,2729438,10:14:16,10:14:16,22831,3,NaN,NaN,NaN,NaN,NaN,...,101856,-112.005087,NaN,NaN,S 4800 W,4800 W @ 9049 S,0,NaN,POINT (-112.005087 40.586969),22831
3,2729438,10:14:33,10:14:33,23257,4,NaN,NaN,NaN,NaN,NaN,...,198077,-112.005766,NaN,NaN,W 9000 S,9000 S @ 4820 W,0,NaN,POINT (-112.005766 40.588034),23257
4,2729438,10:15:50,10:15:50,23255,5,NaN,NaN,NaN,NaN,NaN,...,198075,-112.013903,NaN,NaN,W 9000 S,9000 S @ 5156 W,0,NaN,POINT (-112.013903 40.588043),23255
5,2729438,10:17:00,10:17:00,23253,6,NaN,NaN,NaN,NaN,NaN,...,198073,-112.014617,NaN,NaN,S GRIZZLY WAY,GRIZZLY WAY @ 8731 S,0,NaN,POINT (-112.014617 40.592874),23253
6,2729438,10:18:07,10:18:07,23312,7,NaN,NaN,NaN,NaN,NaN,...,198091,-112.017180,NaN,NaN,S GRIZZLY WAY,GRIZZLY WAY @ 8461 S,0,NaN,POINT (-112.01718 40.597704),23312
7,2729438,10:19:00,10:19:00,23250,8,NaN,NaN,NaN,NaN,NaN,...,198070,-112.019933,NaN,NaN,S GRIZZLY WAY,GRIZZLY WAY @ 8271 S,0,NaN,POINT (-112.019933 40.60083),23250
8,2729438,10:20:16,10:20:16,23248,9,NaN,NaN,NaN,NaN,NaN,...,198068,-112.013517,NaN,NaN,S GRIZZLY WAY,GRIZZLY WAY @ 7981 S,0,NaN,POINT (-112.013517 40.606621),23248
9,2729438,10:20:51,10:20:51,23247,10,NaN,NaN,NaN,NaN,NaN,...,198067,-112.012559,NaN,NaN,S GRIZZLY WAY,GRIZZLY WAY @ 7761 S,0,NaN,POINT (-112.012559 40.61035),23247
